<a href="https://colab.research.google.com/github/yedini/DL_tensorflow/blob/main/Part2_Lab03_Relu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.5.0


#### Checkpoint function

In [ ]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

#### Data load & pre-processing function

In [ ]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()

    # tensorflow input은 [batch_size, height, width, channel] 형태여야함
    # but mnist는 grey scale이라 channel이 없으므로 채널을 만들어줌(shape을 맞춰줌)
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    # 10: 데이터 라벨의 개수
    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

# normalization: 이미지가 갖는 숫자값의 범위: 0~255 => 255로 나눠준다.
def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

#### Performance function

In [ ]:
def loss_fn(model, images, labels):
    # input을 model에 넣어서 output을 logit에 넣음.
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

#### Model function

In [ ]:
def flatten() :   #shape을 펼쳐주는 역할
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :   #fully connected layer
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

#### Create model

In [ ]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    # 네트워크에 layer를 쌓는 것: list에 계속 layer를 더해주는 것과 같음.
    # Sequential은 list의 자료구조 타입이라고 볼 수 있다.
    model = tf.keras.Sequential()
    model.add(flatten())  #fully connected model을 사용하기 때문에 flatten 해준 것!

    for i in range(2) :  #fully connected와 relu를 두 번씩 -> 총 네번 사용
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))  #최종 output 

    return model

#### **model class version

In [ ]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

#### Define data & hyper-parameter

In [ ]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
## dataset API를 이용해서 이미지와 라벨들(data)을 네트워크에 넣는다.
# shuffle: 데이터 섞기 - size는 input data보다 커야함
# prefetch: 네트워크가 batch size만큼 학습하고 있을 때,
#           미리 메모리에 batch size만큼 올려놓음 => 속도가 빨라진다
#batch: batch-size만큼 네트워크에 넣는다.
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

11493376/11490434 [==============================] - 0s 0us/step


#### define model & optimizer & writer

In [ ]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

#### Restore checkpoint & start train or test phase

In [ ]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.1669, train_loss: 2.14287925, train_accuracy: 0.4062, test_Accuracy: 0.2887
Epoch: [ 0] [    1/  468] time: 1.3864, train_loss: 2.12167335, train_accuracy: 0.4062, test_Accuracy: 0.4142
Epoch: [ 0] [    2/  468] time: 1.6085, train_loss: 2.04032660, train_accuracy: 0.5312, test_Accuracy: 0.5092
Epoch: [ 0] [    3/  468] time: 1.8637, train_loss: 1.96679568, train_accuracy: 0.6250, test_Accuracy: 0.5765
Epoch: [ 0] [    4/  468] time: 2.0894, train_loss: 1.87393558, train_accuracy: 0.6172, test_Accuracy: 0.6324
Epoch: [ 0] [    5/  468] time: 2.3143, train_loss: 1.78363883, train_accuracy: 0.6641, test_Accuracy: 0.6682
Epoch: [ 0] [    6/  468] time: 2.5346, train_loss: 1.62030935, train_accuracy: 0.7109, test_Accuracy: 0.6570
Epoch: [ 0] [    7/  468] time: 2.7626, train_loss: 1.55639958, train_accuracy: 0.6719, test_Accuracy: 0.6373
Epoch: [ 0] [    8/  468] time: 2.9933,